https://medium.com/pylessons/convolutional-neural-networks-cnn-explained-step-by-step-69137a54e5e7 # spoko wprowadzenie
RGB IMAGES RED, GREEN , BLUE channel that are composed together into one image. In other wodls, the image is composed of three images (one for each channel), where each image can store discrete pxels with conventional brightness instnse between 0 and 255

https://medium.com/@RaghavPrabhu/understanding-of-convolutional-neural-network-cnn-deep-learning-99760835f148 # spoko obrazki 


https://towardsdatascience.com/simple-introduction-to-convolutional-neural-networks-cdf8d3077bac !!! fajne

https://algorithmia.com/blog/convolutional-neural-nets-in-pytorch # jakies CNN zrobione w pytorchu
https://adventuresinmachinelearning.com/convolutional-neural-networks-tutorial-in-pytorch/ # CNN w pytorchu 

https://pylessons.com/CNN-tutorial-introduction/ # dużo pythonowych tutoriali.
http://cs231n.github.io/convolutional-networks/ # stanford lessons and nice pictures of multi-layed kernel
https://medium.com/@purnasaigudikandula/a-beginner-intro-to-convolutional-neural-networks-684c5620c2ce # history of cnn 

In [10]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

In [13]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

In [14]:
#The compose function allows for multiple transforms
#transforms.ToTensor() converts our PILImage to a tensor of shape (C x H x W) in the range [0,1]
#transforms.Normalize(mean,std) normalizes a tensor to a (mean, std) for (R, G, B)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.CIFAR10(root='./cifardata', train=True, download=True, transform=transform)

test_set = torchvision.datasets.CIFAR10(root='./cifardata', train=False, download=True, transform=transform)

0it [00:00, ?it/s]

100%|█████████▉| 170123264/170498071 [01:07<00:00, 2778380.25it/s]

Extracting ./cifardata/cifar-10-python.tar.gz to ./cifardata
Files already downloaded and verified


170500096it [01:20, 2778380.25it/s]                               

In [20]:
#trainloader = torch.utils.data.DataLoader(train_set, batch_size=4,
                                          shuffle=True, num_workers=2)
#testloader = torch.utils.data.DataLoader(test_set, batch_size=4,
                                         shuffle=False, num_workers=2)

In [ ]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [17]:
from torch.utils.data.sampler import SubsetRandomSampler

#Training
n_training_samples = 20000
train_sampler = SubsetRandomSampler(np.arange(n_training_samples, dtype=np.int64))

#Validation
n_val_samples = 5000
val_sampler = SubsetRandomSampler(np.arange(n_training_samples, n_training_samples + n_val_samples, dtype=np.int64))

#Test
n_test_samples = 5000
test_sampler = SubsetRandomSampler(np.arange(n_test_samples, dtype=np.int64))

Writing CNN code in PyTorch can get a little complex, since everything is defined inside of one class. We’ll create a SimpleCNN class, which inherits from the master torch.nn.Module class.

In [24]:
from torch.autograd import Variable
import torch.nn.functional as F

class SimpleCNN(torch.nn.Module):
    
    #Our batch shape for input x is (3, 32, 32)
    
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        #Input channels = 3, output channels = 18
        self.conv1 = torch.nn.Conv2d(3, 18, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        #4608 input features, 64 output features (see sizing flow below)
        self.fc1 = torch.nn.Linear(18 * 16 * 16, 64)
        
        #64 input features, 10 output features for our 10 defined classes
        self.fc2 = torch.nn.Linear(64, 10)
    
    def forward(self, x):
        
        #Computes the activation of the first convolution
        #Size changes from (3, 32, 32) to (18, 32, 32)
        x = F.relu(self.conv1(x))
        
        #Size changes from (18, 32, 32) to (18, 16, 16)
        x = self.pool(x)
        
        #Reshape data to input to the input layer of the neural net
        #Size changes from (18, 16, 16) to (1, 4608)
        #Recall that the -1 infers this dimension from the other given dimension
        x = x.view(-1, 18 * 16 *16)
        
        #Computes the activation of the first fully connected layer
        #Size changes from (1, 4608) to (1, 64)
        x = F.relu(self.fc1(x))
        
        #Computes the second fully connected layer (activation applied later)
        #Size changes from (1, 64) to (1, 10)
        x = self.fc2(x)
        return(x)

SIZE OF OUTPUT AFTER CHANGES 

In [26]:
def outputSize(in_size, kernel_size, stride, padding):
    output = int((in_size - kernel_size + 2*(padding)) / stride) + 1
    return(output)

In [28]:
outputSize(36,2,2,0)

18

In [63]:
#DataLoader takes in a dataset and a sampler for loading (num_workers deals with system level memory) 
def get_train_loader(batch_size):
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                           sampler=train_sampler, num_workers=2)
    return(train_loader)

In [64]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size=4, sampler=test_sampler, num_workers=2)
val_loader = torch.utils.data.DataLoader(train_set, batch_size=128, sampler=val_sampler, num_workers=2)

In [65]:
[test_loader.dataset.data.shape,get_train_loader(4).dataset.data.shape,val_loader.dataset.data.shape]

[(10000, 32, 32, 3), (50000, 32, 32, 3), (50000, 32, 32, 3)]

In [42]:
import torch.optim as optim

def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    loss = torch.nn.CrossEntropyLoss()
    
    #Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

In [68]:
CNN = SimpleCNN()
loss, optimizer= createLossAndOptimizer(CNN)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

### TRAINING OUR NET

In [71]:
import time

def trainNet(net, batch_size, n_epochs, learning_rate):
    
    #Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    #Get training data
    train_loader = get_train_loader(batch_size)
    n_batches = len(train_loader)
    
    #Create our loss and optimizer functions
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(train_loader, 0): # go through the training set with numeration 
            
            #Get inputs
            inputs, labels = data # tensors and labels for them, we know all about that
            
            #Wrap them in a Variable object
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(inputs) # classification 
            loss_size = loss(outputs, labels) # difference in our net and real value of a label #loss = torch.nn.CrossEntropyLoss()
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.data
            total_train_loss += loss_size.data
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        for inputs, labels in val_loader:
            
            #Wrap tensors in Variables
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Forward pass
            val_outputs = net(inputs)
            val_loss_size = loss(val_outputs, labels)
            total_val_loss += val_loss_size.data[0]
            
        print("Validation loss = {:.2f}".format(total_val_loss / len(val_loader)))
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

In [81]:
import time

def trainNet(net, batch_size, n_epochs, learning_rate):
    
    #Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    #Get training data
    train_loader = get_train_loader(batch_size)
    n_batches = len(train_loader)
    
    #Create our loss and optimizer functions
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(train_loader, 0):
            
            #Get inputs
            inputs, labels = data
            
            #Wrap them in a Variable object
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.data
            total_train_loss += loss_size.data
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        for inputs, labels in val_loader:
            
            #Wrap tensors in Variables
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Forward pass
            val_outputs = net(inputs)
            val_loss_size = loss(val_outputs, labels)
            total_val_loss += val_loss_size.data
            
        print("Validation loss = {:.2f}".format(total_val_loss / len(val_loader)))
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

In [82]:
CNN = SimpleCNN()
trainNet(CNN, batch_size=32, n_epochs=5, learning_rate=0.001)

===== HYPERPARAMETERS =====
batch_size= 32
epochs= 5
learning_rate= 0.001
Epoch 1, 10% 	 train_loss: 2.07 took: 1.77s
Epoch 1, 20% 	 train_loss: 1.85 took: 1.59s
Epoch 1, 30% 	 train_loss: 1.74 took: 1.85s
Epoch 1, 40% 	 train_loss: 1.61 took: 1.60s
Epoch 1, 50% 	 train_loss: 1.54 took: 1.58s
Epoch 1, 60% 	 train_loss: 1.48 took: 1.66s
Epoch 1, 70% 	 train_loss: 1.49 took: 1.69s
Epoch 1, 80% 	 train_loss: 1.46 took: 1.59s
Epoch 1, 90% 	 train_loss: 1.46 took: 1.69s
Validation loss = 1.38
Epoch 2, 10% 	 train_loss: 1.34 took: 1.71s
Epoch 2, 20% 	 train_loss: 1.31 took: 1.71s
Epoch 2, 30% 	 train_loss: 1.29 took: 1.83s
Epoch 2, 40% 	 train_loss: 1.32 took: 1.90s
Epoch 2, 50% 	 train_loss: 1.35 took: 1.70s
Epoch 2, 60% 	 train_loss: 1.30 took: 1.69s
Epoch 2, 70% 	 train_loss: 1.25 took: 1.84s
Epoch 2, 80% 	 train_loss: 1.27 took: 1.84s
Epoch 2, 90% 	 train_loss: 1.31 took: 1.71s
Validation loss = 1.27
Epoch 3, 10% 	 train_loss: 1.21 took: 1.82s
Epoch 3, 20% 	 train_loss: 1.15 took: 1.73s


In [83]:
total_val_loss

NameError: name 'total_val_loss' is not defined